# The Battle of Neighborhoods

### London District WC1R

The aim of this project is to explore what venues are available around WC1R. In the first part, I have carried out data processing and kmeans clustering of the venues data. I have aimed to show what venues are least commons in order for this data to be helpful for entrepreneurs looking for an opportunity to set up business in the areas in question. The average income information could further serve as an indicator of the areas' living standards.

<br/> The project deals with data about WC1R district in London. We combine two datasets: 
<br/> 1) doogal.co.uk - dataset with London postcodes with borough, district and average income details. Due to the large number of post codes in London, I have filtered and worked only with post codes within the WC1R London district.
<br/> 2) Foresquare - explore venues around the WC1R London district.


#### Import Libraries

In [155]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

### Download and prepare data about London West End, District WC1R

#### Scrape the List of postal codes and coordinates of WC1R from Doogal and create a DataFrame

In [156]:
# Read csv file with data
df_all_Lnd=pd.read_csv('https://www.doogal.co.uk/UKPostcodesCSV.ashx?region=E12000007')
df_all_Lnd.head()

,Postcode,In Use?,Latitude,Longitude,Easting,Northing,Grid Ref,County,District,Ward,...,User Type,Last updated,Nearest station,Distance to station,Postcode area,Postcode district,Police force,Water company,Plus Code,Average Income
0,BR1 1AA,Yes,51.401546,0.015415,540291,168873,TQ402688,Greater London,Bromley,Bromley Town,...,0,2020-02-19,Bromley South,0.218257,BR,BR1,Metropolitan Police,Thames Water,9F32C228+J5,63100
1,BR1 1AB,Yes,51.406333,0.015208,540262,169405,TQ402694,Greater London,Bromley,Bromley Town,...,0,2020-02-19,Bromley North,0.253666,BR,BR1,Metropolitan Police,Thames Water,9F32C248+G3,56100
2,BR1 1AD,No,51.400057,0.016715,540386,168710,TQ403687,Greater London,Bromley,Bromley Town,...,1,2020-02-19,Bromley South,0.044559,BR,BR1,Metropolitan Police,NaN,9F32C228+2M,63100
3,BR1 1AE,Yes,51.404543,0.014195,540197,169204,TQ401692,Greater London,Bromley,Bromley Town,...,0,2020-02-19,Bromley North,0.462939,BR,BR1,Metropolitan Police,Thames Water,9F32C237+RM,63100
4,BR1 1AF,Yes,51.401392,0.014948,540259,168855,TQ402688,Greater London,Bromley,Bromley Town,...,0,2020-02-19,Bromley South,0.227664,BR,BR1,Metropolitan Police,Thames Water,9F32C227+HX,63100


In [157]:
# Filter column Postcode for post codes of WC1R only
check = df_all_Lnd["Postcode district"]=='WC1R' 
print(df_all_Lnd.shape)
df_all_Lnd[check].head()

(321834, 47)


,Postcode,In Use?,Latitude,Longitude,Easting,Northing,Grid Ref,County,District,Ward,...,User Type,Last updated,Nearest station,Distance to station,Postcode area,Postcode district,Police force,Water company,Plus Code,Average Income
316981,WC1R 4AA,No,51.518266,-0.120144,530529,181602,TQ305816,Greater London,Camden,Holborn and Covent Garden,...,0,2020-02-19,Holborn,0.116391,WC,WC1R,Metropolitan Police,Thames Water,9C3XGV9H+8W,50200
316982,WC1R 4AB,Yes,51.518026,-0.119765,530556,181576,TQ305815,Greater London,Camden,Holborn and Covent Garden,...,0,2020-02-19,Holborn,0.087969,WC,WC1R,Metropolitan Police,Thames Water,9C3XGV9J+63,50200
316983,WC1R 4AD,No,51.518010,-0.119909,530546,181574,TQ305815,Greater London,Camden,Holborn and Covent Garden,...,0,2020-02-19,Holborn,0.086167,WC,WC1R,Metropolitan Police,NaN,9C3XGV9J+62,50200
316984,WC1R 4AE,No,51.519299,-0.117276,530725,181722,TQ307817,Greater London,Camden,Holborn and Covent Garden,...,1,2020-02-19,Holborn,0.289843,WC,WC1R,Metropolitan Police,NaN,9C3XGV9M+P3,50200
316985,WC1R 4AF,No,51.518231,-0.119122,530600,181600,TQ306815,Greater London,Camden,Holborn and Covent Garden,...,0,2020-02-19,Holborn,0.121241,WC,WC1R,Metropolitan Police,NaN,9C3XGV9J+79,50200


In [158]:
# Clean DataFrame
df_WC_Lnd=df_all_Lnd[check]
df_WC_Lnd.reset_index(inplace=True, drop=True)
df_WC_Lnd.drop(columns=['In Use?', 'Easting', 'Northing', 'Grid Ref', 'District Code', 'Ward Code',
       'Country', 'County Code', 'Introduced', 'Terminated',
       'Parish', 'National Park', 'Population', 'Households', 'Built up area',
       'Built up sub-division', 'Lower layer super output area', 'Rural/urban',
       'Region', 'Altitude', 'London zone', 'LSOA Code', 'Local authority',
       'MSOA Code', 'Middle layer super output area', 'Parish Code',
       'Census output area', 'Constituency Code',
       'Index of Multiple Deprivation', 'Quality', 'User Type', 'Last updated',
       'Nearest station', 'Distance to station', 'Police force', 'Water company', 'Plus Code'], inplace=True)
df_WC_Lnd.rename(columns={'Latitude':'Latitude_Lnd',
                          'Longitude':'Longitude_Lnd',
                          'Postcode':'PostalCode_Lnd', 'Ward':'Area'}, 
                 inplace=True)
print(df_WC_Lnd.shape)
df_WC_Lnd.head()

(330, 10)


/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,PostalCode_Lnd,Latitude_Lnd,Longitude_Lnd,County,District,Area,Constituency,Postcode area,Postcode district,Average Income
0,WC1R 4AA,51.518266,-0.120144,Greater London,Camden,Holborn and Covent Garden,Holborn and St Pancras,WC,WC1R,50200
1,WC1R 4AB,51.518026,-0.119765,Greater London,Camden,Holborn and Covent Garden,Holborn and St Pancras,WC,WC1R,50200
2,WC1R 4AD,51.518010,-0.119909,Greater London,Camden,Holborn and Covent Garden,Holborn and St Pancras,WC,WC1R,50200
3,WC1R 4AE,51.519299,-0.117276,Greater London,Camden,Holborn and Covent Garden,Holborn and St Pancras,WC,WC1R,50200
4,WC1R 4AF,51.518231,-0.119122,Greater London,Camden,Holborn and Covent Garden,Holborn and St Pancras,WC,WC1R,50200


#### Explore and cluster the Neighborhoods in WC1R District of London

Prepare map facilities

In [159]:
!pip install geopy
from geopy.geocoders import Nominatim 

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode('WC1R')
latitude_L = location.latitude
longitude_L = location.longitude
print('The geograpical coordinate of London District WC1R are {}, {}.'.format(latitude_L, longitude_L))

The geograpical coordinate of London District WC1R are 51.5191352, -0.1154919.


In [160]:
!pip install folium
import folium # map rendering library

Create a map of London

In [161]:
# Create map of London using latitude_L and longitude_L values
map_WestLondon = folium.Map(location=[latitude_L, longitude_L], zoom_start=17)

# Add markers to map
for lat, lng, label in zip(df_WC_Lnd['Latitude_Lnd'], df_WC_Lnd['Longitude_Lnd'], df_WC_Lnd['Area']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_WestLondon)  
    
map_WestLondon

#### Venues in west London
From Foursquare, extract information about venues around WC1R District, London based on the coordinates in df_coord_Lnd .

In [46]:
# The code was removed by Watson Studio for sharing.

In [47]:
radius=500
LIMIT=100

In [75]:
# Set up a function to extract Londonb WC1R nearby venues
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'], 
            v['venue']['location']['postalCode'],
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Area', 
                  'Area Latitude', 
                  'Area Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',
                  'PostalCode_Venue',
                  'Venue Category']
    
    return(nearby_venues)

In [76]:
# Extract venues nearby London
london_venues = getNearbyVenues(names=df_WC_Lnd['Area'],
                                   latitudes=df_WC_Lnd['Latitude_Lnd'],
                                   longitudes=df_WC_Lnd['Longitude_Lnd']
                                  )

Holborn and Covent Garden


KeyError: 'groups'

In [77]:
# Display DataFrame with venues
london_venues

,Area,Latitude_Lnd,Longitude_Lnd,Venue,Venue Latitude,Venue Longitude,PostalCode_Venue,Venue Category
0,Holborn and Covent Garden,51.518266,-0.120144,The Espresso Room,51.518399,-0.120824,WC1B 5HA,Coffee Shop
1,Holborn and Covent Garden,51.518026,-0.119765,Gymbox,51.517952,-0.119089,WC1V 6RD,Gym / Fitness Center
2,Holborn and Covent Garden,51.518010,-0.119909,L'Oscar,51.517988,-0.120199,WC1B 4AA,Hotel
3,Holborn and Covent Garden,51.519299,-0.117276,Kozzy,51.519659,-0.117232,WC1R 4PD,Sandwich Place
4,Holborn and Covent Garden,51.518231,-0.119122,Gymbox,51.517952,-0.119089,WC1V 6RD,Gym / Fitness Center
5,Holborn and Covent Garden,51.519738,-0.118858,Conway Hall,51.519991,-0.118266,WC1R 4RL,Convention Center
6,Holborn and Covent Garden,51.519738,-0.118858,Conway Hall,51.519991,-0.118266,WC1R 4RL,Convention Center
7,Holborn and Covent Garden,51.518773,-0.117039,Sawadika,51.518480,-0.116885,WC1R 4NA,Thai Restaurant
8,Holborn and Covent Garden,51.518652,-0.117317,Sawadika,51.518480,-0.116885,WC1R 4NA,Thai Restaurant
9,Holborn and Covent Garden,51.518652,-0.117317,Sawadika,51.518480,-0.116885,WC1R 4NA,Thai Restaurant


In [162]:
london_venues.shape

(316, 8)

In [163]:
london_venues.drop(london_venues[london_venues['PostalCode_Venue']=="NaN"].index,axis=0, inplace=True)

In [164]:
london_venues.shape

(316, 8)

In [417]:
# Display numbers of venues for each postcode
london_venues.groupby('PostalCode_Venue').count()

,Area,Latitude_Lnd,Longitude_Lnd,Venue,Venue Latitude,Venue Longitude,Venue Category
PostalCode_Venue,,,,,,,
W C1R,11,11,11,11,11,11,11
W1T 1HT,16,16,16,16,16,16,16
WC1,21,21,21,21,21,21,21
WC1B 4AA,2,2,2,2,2,2,2
WC1B 5HA,1,1,1,1,1,1,1
WC1R 4LR,28,28,28,28,28,28,28
WC1R 4NA,17,17,17,17,17,17,17
WC1R 4PD,76,76,76,76,76,76,76
WC1R 4RL,6,6,6,6,6,6,6


#### One Hot Encoding 

In [167]:
london_onehot = pd.get_dummies(london_venues[['Venue Category']], prefix="", prefix_sep="")

# Add area column back to dataframe
london_onehot['PostalCode_Venue'] = london_venues['PostalCode_Venue'] 
london_onehot['PostalCode_Venue'] = london_venues['PostalCode_Venue'] 

# Move neighborhood column to the first column
fixed_columns = [london_onehot.columns[-1]] + list(london_onehot.columns[:-1])
london_onehot.head()

,Argentinian Restaurant,Café,Clothing Store,Coffee Shop,Convention Center,Garden,Gluten-free Restaurant,Gym / Fitness Center,Hotel,Italian Restaurant,...,Optical Shop,Persian Restaurant,Pharmacy,Plaza,Pub,Sandwich Place,Sushi Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,PostalCode_Venue
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,WC1B 5HA
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,WC1V 6RD
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,WC1B 4AA
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,WC1R 4PD
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,WC1V 6RD


In [264]:
# Group onehot results by PostalCode_Venue
london_grouped = london_onehot.groupby('PostalCode_Venue').mean().reset_index()
london_grouped

,PostalCode_Venue,Argentinian Restaurant,Café,Clothing Store,Coffee Shop,Convention Center,Garden,Gluten-free Restaurant,Gym / Fitness Center,Hotel,...,Office,Optical Shop,Persian Restaurant,Pharmacy,Plaza,Pub,Sandwich Place,Sushi Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant
0,W C1R,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
1,W1T 1HT,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
2,WC1,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,1.0,0.000000,0.000000,0.0,0.0,0.0
3,WC1B 4AA,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,1.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
4,WC1B 5HA,0.000000,0.0,0.000000,1.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
5,WC1R 4LR,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,1.0,0.0,0.0
6,WC1R 4NA,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,1.0,0.0
7,WC1R 4PD,0.157895,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.842105,0.0,0.0,0.0
8,WC1R 4RL,0.000000,0.0,0.000000,0.0,1.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
9,WC1V 6AE,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0


In [263]:
# Calculate frequency of venues based on their number per PostalCode_Venue and display top 5
num_top_venues = 5

for hood in london_grouped['PostalCode_Venue']:
    print("----"+hood+"----")
    temp = london_grouped[london_grouped['PostalCode_Venue'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----W C1R----
                           venue     freq
0                 Average Income  50200.0
1             Italian Restaurant      1.0
2                         Office      0.0
3  Vegetarian / Vegan Restaurant      0.0
4                Thai Restaurant      0.0


----W1T 1HT----
                           venue     freq
0                 Average Income  50200.0
1                      Juice Bar      1.0
2                 Clothing Store      0.0
3                   Optical Shop      0.0
4  Vegetarian / Vegan Restaurant      0.0


----WC1----
                           venue     freq
0                 Average Income  50200.0
1                          Plaza      1.0
2                         Office      0.0
3  Vegetarian / Vegan Restaurant      0.0
4                Thai Restaurant      0.0


----WC1B 4AA----
                           venue     freq
0                 Average Income  50200.0
1                          Hotel      1.0
2                         Office      0.0
3  Vegetari

In [265]:
# Sort in Ascending order to bring least frequent ones on top
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=True)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
The below DataFrame areas_venues_sorted displays a ranking of the least common types of venues within the WC1R district of London.

In [266]:
# Create a DataFrame showing ranking venues categories based on frequency per neighborhood
num_top_venues = 22

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode_Venue']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Least Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Least Common Venue'.format(ind+1))

# create a new dataframe
areas_venues_sorted = pd.DataFrame(columns=columns)
areas_venues_sorted['PostalCode_Venue'] = london_grouped['PostalCode_Venue']

for ind in np.arange(london_grouped.shape[0]):
    areas_venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

areas_venues_sorted

,PostalCode_Venue,1st Least Common Venue,2nd Least Common Venue,3rd Least Common Venue,4th Least Common Venue,5th Least Common Venue,6th Least Common Venue,7th Least Common Venue,8th Least Common Venue,9th Least Common Venue,...,13th Least Common Venue,14th Least Common Venue,15th Least Common Venue,16th Least Common Venue,17th Least Common Venue,18th Least Common Venue,19th Least Common Venue,20th Least Common Venue,21th Least Common Venue,22th Least Common Venue
0,W C1R,Argentinian Restaurant,Sushi Restaurant,Sandwich Place,Pub,Plaza,Pharmacy,Persian Restaurant,Optical Shop,Office,...,Gym / Fitness Center,Gluten-free Restaurant,Garden,Convention Center,Coffee Shop,Clothing Store,Café,Thai Restaurant,Vegetarian / Vegan Restaurant,Italian Restaurant
1,W1T 1HT,Argentinian Restaurant,Sushi Restaurant,Sandwich Place,Pub,Plaza,Pharmacy,Persian Restaurant,Optical Shop,Office,...,Gym / Fitness Center,Gluten-free Restaurant,Garden,Convention Center,Coffee Shop,Clothing Store,Café,Italian Restaurant,Vegetarian / Vegan Restaurant,Juice Bar
2,WC1,Argentinian Restaurant,Sushi Restaurant,Sandwich Place,Pub,Pharmacy,Persian Restaurant,Optical Shop,Office,Juice Bar,...,Gym / Fitness Center,Gluten-free Restaurant,Garden,Convention Center,Coffee Shop,Clothing Store,Café,Italian Restaurant,Vegetarian / Vegan Restaurant,Plaza
3,WC1B 4AA,Argentinian Restaurant,Sushi Restaurant,Sandwich Place,Pub,Plaza,Pharmacy,Persian Restaurant,Optical Shop,Office,...,Gym / Fitness Center,Gluten-free Restaurant,Garden,Convention Center,Coffee Shop,Clothing Store,Café,Thai Restaurant,Vegetarian / Vegan Restaurant,Hotel
4,WC1B 5HA,Argentinian Restaurant,Sushi Restaurant,Sandwich Place,Pub,Plaza,Pharmacy,Persian Restaurant,Optical Shop,Office,...,Hotel,Gym / Fitness Center,Gluten-free Restaurant,Garden,Convention Center,Clothing Store,Café,Thai Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop
5,WC1R 4LR,Argentinian Restaurant,Sandwich Place,Pub,Plaza,Pharmacy,Persian Restaurant,Optical Shop,Office,Juice Bar,...,Gym / Fitness Center,Gluten-free Restaurant,Garden,Convention Center,Coffee Shop,Clothing Store,Café,Italian Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant
6,WC1R 4NA,Argentinian Restaurant,Sushi Restaurant,Sandwich Place,Pub,Plaza,Pharmacy,Persian Restaurant,Optical Shop,Office,...,Hotel,Gym / Fitness Center,Gluten-free Restaurant,Garden,Convention Center,Coffee Shop,Clothing Store,Café,Vegetarian / Vegan Restaurant,Thai Restaurant
7,WC1R 4PD,Japanese Restaurant,Sushi Restaurant,Pub,Plaza,Pharmacy,Persian Restaurant,Optical Shop,Office,Juice Bar,...,Gym / Fitness Center,Gluten-free Restaurant,Garden,Convention Center,Coffee Shop,Clothing Store,Café,Italian Restaurant,Argentinian Restaurant,Sandwich Place
8,WC1R 4RL,Argentinian Restaurant,Sushi Restaurant,Sandwich Place,Pub,Plaza,Pharmacy,Persian Restaurant,Optical Shop,Office,...,Hotel,Gym / Fitness Center,Gluten-free Restaurant,Garden,Coffee Shop,Clothing Store,Café,Thai Restaurant,Vegetarian / Vegan Restaurant,Convention Center
9,WC1V 6AE,Argentinian Restaurant,Sushi Restaurant,Sandwich Place,Pub,Plaza,Pharmacy,Persian Restaurant,Optical Shop,Office,...,Hotel,Gym / Fitness Center,Gluten-free Restaurant,Garden,Convention Center,Coffee Shop,Clothing Store,Café,Thai Restaurant,Japanese Restaurant


In [267]:
areas_venues_sorted.shape

(23, 23)

### Cluster Postal Codes

The below kmeans analysis is similar to the one carried out with the New York and Toronto data sets. It is based on the information available on Foursquare about venues within the WC1R district of London.

In [268]:
# Use k-means to cluster the areas into 4 clusters
from sklearn.cluster import KMeans 

# Group neighbourhoods and venues data into 5 clusters using k-means
kclusters = 4

london_grouped_clustering = london_grouped.drop('PostalCode_Venue', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 
# to change use .astype()

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 3, 0, 0, 1, 0, 0, 0,
       3], dtype=int32)

In [269]:
# add clustering labels
areas_venues_sorted.insert(0, 'Cluster_Labels', kmeans.labels_)


In [270]:
print(areas_venues_sorted.shape)
areas_venues_sorted

(23, 24)


,Cluster_Labels,PostalCode_Venue,1st Least Common Venue,2nd Least Common Venue,3rd Least Common Venue,4th Least Common Venue,5th Least Common Venue,6th Least Common Venue,7th Least Common Venue,8th Least Common Venue,...,13th Least Common Venue,14th Least Common Venue,15th Least Common Venue,16th Least Common Venue,17th Least Common Venue,18th Least Common Venue,19th Least Common Venue,20th Least Common Venue,21th Least Common Venue,22th Least Common Venue
0,0,W C1R,Argentinian Restaurant,Sushi Restaurant,Sandwich Place,Pub,Plaza,Pharmacy,Persian Restaurant,Optical Shop,...,Gym / Fitness Center,Gluten-free Restaurant,Garden,Convention Center,Coffee Shop,Clothing Store,Café,Thai Restaurant,Vegetarian / Vegan Restaurant,Italian Restaurant
1,0,W1T 1HT,Argentinian Restaurant,Sushi Restaurant,Sandwich Place,Pub,Plaza,Pharmacy,Persian Restaurant,Optical Shop,...,Gym / Fitness Center,Gluten-free Restaurant,Garden,Convention Center,Coffee Shop,Clothing Store,Café,Italian Restaurant,Vegetarian / Vegan Restaurant,Juice Bar
2,0,WC1,Argentinian Restaurant,Sushi Restaurant,Sandwich Place,Pub,Pharmacy,Persian Restaurant,Optical Shop,Office,...,Gym / Fitness Center,Gluten-free Restaurant,Garden,Convention Center,Coffee Shop,Clothing Store,Café,Italian Restaurant,Vegetarian / Vegan Restaurant,Plaza
3,0,WC1B 4AA,Argentinian Restaurant,Sushi Restaurant,Sandwich Place,Pub,Plaza,Pharmacy,Persian Restaurant,Optical Shop,...,Gym / Fitness Center,Gluten-free Restaurant,Garden,Convention Center,Coffee Shop,Clothing Store,Café,Thai Restaurant,Vegetarian / Vegan Restaurant,Hotel
4,1,WC1B 5HA,Argentinian Restaurant,Sushi Restaurant,Sandwich Place,Pub,Plaza,Pharmacy,Persian Restaurant,Optical Shop,...,Hotel,Gym / Fitness Center,Gluten-free Restaurant,Garden,Convention Center,Clothing Store,Café,Thai Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop
5,0,WC1R 4LR,Argentinian Restaurant,Sandwich Place,Pub,Plaza,Pharmacy,Persian Restaurant,Optical Shop,Office,...,Gym / Fitness Center,Gluten-free Restaurant,Garden,Convention Center,Coffee Shop,Clothing Store,Café,Italian Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant
6,0,WC1R 4NA,Argentinian Restaurant,Sushi Restaurant,Sandwich Place,Pub,Plaza,Pharmacy,Persian Restaurant,Optical Shop,...,Hotel,Gym / Fitness Center,Gluten-free Restaurant,Garden,Convention Center,Coffee Shop,Clothing Store,Café,Vegetarian / Vegan Restaurant,Thai Restaurant
7,0,WC1R 4PD,Japanese Restaurant,Sushi Restaurant,Pub,Plaza,Pharmacy,Persian Restaurant,Optical Shop,Office,...,Gym / Fitness Center,Gluten-free Restaurant,Garden,Convention Center,Coffee Shop,Clothing Store,Café,Italian Restaurant,Argentinian Restaurant,Sandwich Place
8,0,WC1R 4RL,Argentinian Restaurant,Sushi Restaurant,Sandwich Place,Pub,Plaza,Pharmacy,Persian Restaurant,Optical Shop,...,Hotel,Gym / Fitness Center,Gluten-free Restaurant,Garden,Coffee Shop,Clothing Store,Café,Thai Restaurant,Vegetarian / Vegan Restaurant,Convention Center
9,2,WC1V 6AE,Argentinian Restaurant,Sushi Restaurant,Sandwich Place,Pub,Plaza,Pharmacy,Persian Restaurant,Optical Shop,...,Hotel,Gym / Fitness Center,Gluten-free Restaurant,Garden,Convention Center,Coffee Shop,Clothing Store,Café,Thai Restaurant,Japanese Restaurant


In [271]:
london_merged = london_venues

# merge london_grouped with areas_venues_sorted to add latitude/longitude for each neighborhood
london_merged = london_merged.join(areas_venues_sorted.set_index('PostalCode_Venue'), on='PostalCode_Venue')

In [272]:
print(london_merged.shape)
london_merged.head()

(316, 31)


,Area,Latitude_Lnd,Longitude_Lnd,Venue,Venue Latitude,Venue Longitude,PostalCode_Venue,Venue Category,Cluster_Labels,1st Least Common Venue,...,13th Least Common Venue,14th Least Common Venue,15th Least Common Venue,16th Least Common Venue,17th Least Common Venue,18th Least Common Venue,19th Least Common Venue,20th Least Common Venue,21th Least Common Venue,22th Least Common Venue
0,Holborn and Covent Garden,51.518266,-0.120144,The Espresso Room,51.518399,-0.120824,WC1B 5HA,Coffee Shop,1,Argentinian Restaurant,...,Hotel,Gym / Fitness Center,Gluten-free Restaurant,Garden,Convention Center,Clothing Store,Café,Thai Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop
1,Holborn and Covent Garden,51.518026,-0.119765,Gymbox,51.517952,-0.119089,WC1V 6RD,Gym / Fitness Center,0,Argentinian Restaurant,...,Hotel,Gluten-free Restaurant,Garden,Convention Center,Coffee Shop,Clothing Store,Café,Thai Restaurant,Vegetarian / Vegan Restaurant,Gym / Fitness Center
2,Holborn and Covent Garden,51.518010,-0.119909,L'Oscar,51.517988,-0.120199,WC1B 4AA,Hotel,0,Argentinian Restaurant,...,Gym / Fitness Center,Gluten-free Restaurant,Garden,Convention Center,Coffee Shop,Clothing Store,Café,Thai Restaurant,Vegetarian / Vegan Restaurant,Hotel
3,Holborn and Covent Garden,51.519299,-0.117276,Kozzy,51.519659,-0.117232,WC1R 4PD,Sandwich Place,0,Japanese Restaurant,...,Gym / Fitness Center,Gluten-free Restaurant,Garden,Convention Center,Coffee Shop,Clothing Store,Café,Italian Restaurant,Argentinian Restaurant,Sandwich Place
4,Holborn and Covent Garden,51.518231,-0.119122,Gymbox,51.517952,-0.119089,WC1V 6RD,Gym / Fitness Center,0,Argentinian Restaurant,...,Hotel,Gluten-free Restaurant,Garden,Convention Center,Coffee Shop,Clothing Store,Café,Thai Restaurant,Vegetarian / Vegan Restaurant,Gym / Fitness Center


In [273]:
london_merged=london_merged.dropna()

In [274]:
london_merged['Cluster_Labels'] = london_merged.Cluster_Labels.astype(int)

In [276]:
print(london_merged.shape)
london_merged.head()

(316, 31)


,Area,Latitude_Lnd,Longitude_Lnd,Venue,Venue Latitude,Venue Longitude,PostalCode_Venue,Venue Category,Cluster_Labels,1st Least Common Venue,...,13th Least Common Venue,14th Least Common Venue,15th Least Common Venue,16th Least Common Venue,17th Least Common Venue,18th Least Common Venue,19th Least Common Venue,20th Least Common Venue,21th Least Common Venue,22th Least Common Venue
0,Holborn and Covent Garden,51.518266,-0.120144,The Espresso Room,51.518399,-0.120824,WC1B 5HA,Coffee Shop,1,Argentinian Restaurant,...,Hotel,Gym / Fitness Center,Gluten-free Restaurant,Garden,Convention Center,Clothing Store,Café,Thai Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop
1,Holborn and Covent Garden,51.518026,-0.119765,Gymbox,51.517952,-0.119089,WC1V 6RD,Gym / Fitness Center,0,Argentinian Restaurant,...,Hotel,Gluten-free Restaurant,Garden,Convention Center,Coffee Shop,Clothing Store,Café,Thai Restaurant,Vegetarian / Vegan Restaurant,Gym / Fitness Center
2,Holborn and Covent Garden,51.518010,-0.119909,L'Oscar,51.517988,-0.120199,WC1B 4AA,Hotel,0,Argentinian Restaurant,...,Gym / Fitness Center,Gluten-free Restaurant,Garden,Convention Center,Coffee Shop,Clothing Store,Café,Thai Restaurant,Vegetarian / Vegan Restaurant,Hotel
3,Holborn and Covent Garden,51.519299,-0.117276,Kozzy,51.519659,-0.117232,WC1R 4PD,Sandwich Place,0,Japanese Restaurant,...,Gym / Fitness Center,Gluten-free Restaurant,Garden,Convention Center,Coffee Shop,Clothing Store,Café,Italian Restaurant,Argentinian Restaurant,Sandwich Place
4,Holborn and Covent Garden,51.518231,-0.119122,Gymbox,51.517952,-0.119089,WC1V 6RD,Gym / Fitness Center,0,Argentinian Restaurant,...,Hotel,Gluten-free Restaurant,Garden,Convention Center,Coffee Shop,Clothing Store,Café,Thai Restaurant,Vegetarian / Vegan Restaurant,Gym / Fitness Center


#### Create a map to visualize venues per clusters

In [291]:
# Import Matplotlib and associated plotting modules
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[latitude_L, longitude_L], zoom_start=15)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_merged['Venue Latitude'], london_merged['Venue Longitude'], london_merged['Area'], london_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine data based one Venues and Average Income

Add Average income per postal code from df_WC_Lnd

In [278]:
london_merged.sort_values(by=['Latitude_Lnd'])
df_WC_Lnd.sort_values(by=['Latitude_Lnd'])

london_merged_revenue = pd.concat([london_merged, df_WC_Lnd], axis=1)
london_merged_revenue.dropna(subset = ["Venue"], inplace=True)
print(london_merged_revenue.shape)
london_merged_revenue

(316, 41)


,Area,Latitude_Lnd,Longitude_Lnd,Venue,Venue Latitude,Venue Longitude,PostalCode_Venue,Venue Category,Cluster_Labels,1st Least Common Venue,...,PostalCode_Lnd,Latitude_Lnd,Longitude_Lnd,County,District,Area,Constituency,Postcode area,Postcode district,Average Income
0,Holborn and Covent Garden,51.518266,-0.120144,The Espresso Room,51.518399,-0.120824,WC1B 5HA,Coffee Shop,1.0,Argentinian Restaurant,...,WC1R 4AA,51.518266,-0.120144,Greater London,Camden,Holborn and Covent Garden,Holborn and St Pancras,WC,WC1R,50200
1,Holborn and Covent Garden,51.518026,-0.119765,Gymbox,51.517952,-0.119089,WC1V 6RD,Gym / Fitness Center,0.0,Argentinian Restaurant,...,WC1R 4AB,51.518026,-0.119765,Greater London,Camden,Holborn and Covent Garden,Holborn and St Pancras,WC,WC1R,50200
2,Holborn and Covent Garden,51.518010,-0.119909,L'Oscar,51.517988,-0.120199,WC1B 4AA,Hotel,0.0,Argentinian Restaurant,...,WC1R 4AD,51.518010,-0.119909,Greater London,Camden,Holborn and Covent Garden,Holborn and St Pancras,WC,WC1R,50200
3,Holborn and Covent Garden,51.519299,-0.117276,Kozzy,51.519659,-0.117232,WC1R 4PD,Sandwich Place,0.0,Japanese Restaurant,...,WC1R 4AE,51.519299,-0.117276,Greater London,Camden,Holborn and Covent Garden,Holborn and St Pancras,WC,WC1R,50200
4,Holborn and Covent Garden,51.518231,-0.119122,Gymbox,51.517952,-0.119089,WC1V 6RD,Gym / Fitness Center,0.0,Argentinian Restaurant,...,WC1R 4AF,51.518231,-0.119122,Greater London,Camden,Holborn and Covent Garden,Holborn and St Pancras,WC,WC1R,50200
5,Holborn and Covent Garden,51.519738,-0.118858,Conway Hall,51.519991,-0.118266,WC1R 4RL,Convention Center,0.0,Argentinian Restaurant,...,WC1R 4AG,51.519738,-0.118858,Greater London,Camden,Holborn and Covent Garden,Holborn and St Pancras,WC,WC1R,50200
6,Holborn and Covent Garden,51.519738,-0.118858,Conway Hall,51.519991,-0.118266,WC1R 4RL,Convention Center,0.0,Argentinian Restaurant,...,WC1R 4AH,51.518343,-0.118253,Greater London,Camden,Holborn and Covent Garden,Holborn and St Pancras,WC,WC1R,50200
7,Holborn and Covent Garden,51.518773,-0.117039,Sawadika,51.518480,-0.116885,WC1R 4NA,Thai Restaurant,0.0,Argentinian Restaurant,...,WC1R 4AJ,51.518773,-0.117039,Greater London,Camden,Holborn and Covent Garden,Holborn and St Pancras,WC,WC1R,50200
8,Holborn and Covent Garden,51.518652,-0.117317,Sawadika,51.518480,-0.116885,WC1R 4NA,Thai Restaurant,0.0,Argentinian Restaurant,...,WC1R 4AL,51.518652,-0.117317,Greater London,Camden,Holborn and Covent Garden,Holborn and St Pancras,WC,WC1R,50200
9,Holborn and Covent Garden,51.518652,-0.117317,Sawadika,51.518480,-0.116885,WC1R 4NA,Thai Restaurant,0.0,Argentinian Restaurant,...,WC1R 4AN,51.518208,-0.117682,Greater London,Camden,Holborn and Covent Garden,Holborn and St Pancras,WC,WC1R,50200


In [279]:
# Display again areas_venues_sorted
print(areas_venues_sorted.shape)
areas_venues_sorted.head()

(23, 24)


,Cluster_Labels,PostalCode_Venue,1st Least Common Venue,2nd Least Common Venue,3rd Least Common Venue,4th Least Common Venue,5th Least Common Venue,6th Least Common Venue,7th Least Common Venue,8th Least Common Venue,...,13th Least Common Venue,14th Least Common Venue,15th Least Common Venue,16th Least Common Venue,17th Least Common Venue,18th Least Common Venue,19th Least Common Venue,20th Least Common Venue,21th Least Common Venue,22th Least Common Venue
0,0,W C1R,Argentinian Restaurant,Sushi Restaurant,Sandwich Place,Pub,Plaza,Pharmacy,Persian Restaurant,Optical Shop,...,Gym / Fitness Center,Gluten-free Restaurant,Garden,Convention Center,Coffee Shop,Clothing Store,Café,Thai Restaurant,Vegetarian / Vegan Restaurant,Italian Restaurant
1,0,W1T 1HT,Argentinian Restaurant,Sushi Restaurant,Sandwich Place,Pub,Plaza,Pharmacy,Persian Restaurant,Optical Shop,...,Gym / Fitness Center,Gluten-free Restaurant,Garden,Convention Center,Coffee Shop,Clothing Store,Café,Italian Restaurant,Vegetarian / Vegan Restaurant,Juice Bar
2,0,WC1,Argentinian Restaurant,Sushi Restaurant,Sandwich Place,Pub,Pharmacy,Persian Restaurant,Optical Shop,Office,...,Gym / Fitness Center,Gluten-free Restaurant,Garden,Convention Center,Coffee Shop,Clothing Store,Café,Italian Restaurant,Vegetarian / Vegan Restaurant,Plaza
3,0,WC1B 4AA,Argentinian Restaurant,Sushi Restaurant,Sandwich Place,Pub,Plaza,Pharmacy,Persian Restaurant,Optical Shop,...,Gym / Fitness Center,Gluten-free Restaurant,Garden,Convention Center,Coffee Shop,Clothing Store,Café,Thai Restaurant,Vegetarian / Vegan Restaurant,Hotel
4,1,WC1B 5HA,Argentinian Restaurant,Sushi Restaurant,Sandwich Place,Pub,Plaza,Pharmacy,Persian Restaurant,Optical Shop,...,Hotel,Gym / Fitness Center,Gluten-free Restaurant,Garden,Convention Center,Clothing Store,Café,Thai Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop


In [286]:
# Add PostalCode_Lnd Average Income to areas_venues_sorted
areas_venues_sorted['Average Income']=london_merged_revenue['Average Income']
areas_venues_sorted['PostalCode_Lnd']=london_merged_revenue['PostalCode_Lnd']

# Sort by Average Income to show PostalCode_Lnd with highest figures on top
areas_venues_sorted=areas_venues_sorted.sort_values(by='Average Income', ascending=False)

# Rearrange columns order to aid analysis
#df = df[['x','y','a','b']]
areas_venues_sorted=areas_venues_sorted[['Cluster_Labels', 'PostalCode_Lnd', 'Average Income', 'PostalCode_Venue', '1st Least Common Venue',
       '2nd Least Common Venue', '3rd Least Common Venue',
       '4th Least Common Venue', '5th Least Common Venue',
       '6th Least Common Venue', '7th Least Common Venue',
       '8th Least Common Venue', '9th Least Common Venue',
       '10th Least Common Venue', '11th Least Common Venue',
       '12th Least Common Venue', '13th Least Common Venue',
       '14th Least Common Venue', '15th Least Common Venue',
       '16th Least Common Venue', '17th Least Common Venue',
       '18th Least Common Venue', '19th Least Common Venue',
       '20th Least Common Venue', '21th Least Common Venue',
       '22th Least Common Venue']]

### Least Common Venues and Income per Postal Code

The below table contains data about least common venues per postal code of WC1R. If an entrepreneur is looking to set up a business in the WC1R district of London, they could refer to the below table and spot an opportunity to open a venue of which there are only few within a chosen post code perimeter. 
Table areas_venues_sorted also provides information of average income per WC1R post code. On this occasion, the data available shows that this income GBP 50,200 across all of the below postal codes, except for WC1R 4AW where it is a bit higher - GBP 54,000. 

Should more diverse data be available about income, this could supplement business location choice further so the entrepreneur place their investments in an area where population protentional has higher purchasing power.

Example Recommendation: If someone is looking to open a restaurant, they should consider opening an Argentinian or Sushi Restaurant, or perhaps a sandwich place (for the latter it would be a good idea to examine the area in person as there might be small shops offering sandwiches which are not registered on Foursquare). All of the below post codes would be appropriate here.

<br/>A particularly good option could be opening a Pub in WC1R 4AG, WC1R 4AG or WC1R 4AZ in these three post codes Pub is the 3rd Least Common venue. in addition, pubs are some of the most popular places to visit in London which makes this a rather good opportunity.


In [288]:
print(areas_venues_sorted.shape)
areas_venues_sorted

(23, 26)


,Cluster_Labels,PostalCode_Lnd,Average Income,PostalCode_Venue,1st Least Common Venue,2nd Least Common Venue,3rd Least Common Venue,4th Least Common Venue,5th Least Common Venue,6th Least Common Venue,...,13th Least Common Venue,14th Least Common Venue,15th Least Common Venue,16th Least Common Venue,17th Least Common Venue,18th Least Common Venue,19th Least Common Venue,20th Least Common Venue,21th Least Common Venue,22th Least Common Venue
16,0,WC1R 4AW,54000,WC1V 7BN,Argentinian Restaurant,Sushi Restaurant,Sandwich Place,Pub,Plaza,Pharmacy,...,Gym / Fitness Center,Gluten-free Restaurant,Garden,Convention Center,Coffee Shop,Clothing Store,Café,Italian Restaurant,Vegetarian / Vegan Restaurant,Optical Shop
1,0,WC1R 4AB,50200,W1T 1HT,Argentinian Restaurant,Sushi Restaurant,Sandwich Place,Pub,Plaza,Pharmacy,...,Gym / Fitness Center,Gluten-free Restaurant,Garden,Convention Center,Coffee Shop,Clothing Store,Café,Italian Restaurant,Vegetarian / Vegan Restaurant,Juice Bar
2,0,WC1R 4AD,50200,WC1,Argentinian Restaurant,Sushi Restaurant,Sandwich Place,Pub,Pharmacy,Persian Restaurant,...,Gym / Fitness Center,Gluten-free Restaurant,Garden,Convention Center,Coffee Shop,Clothing Store,Café,Italian Restaurant,Vegetarian / Vegan Restaurant,Plaza
3,0,WC1R 4AE,50200,WC1B 4AA,Argentinian Restaurant,Sushi Restaurant,Sandwich Place,Pub,Plaza,Pharmacy,...,Gym / Fitness Center,Gluten-free Restaurant,Garden,Convention Center,Coffee Shop,Clothing Store,Café,Thai Restaurant,Vegetarian / Vegan Restaurant,Hotel
4,1,WC1R 4AF,50200,WC1B 5HA,Argentinian Restaurant,Sushi Restaurant,Sandwich Place,Pub,Plaza,Pharmacy,...,Hotel,Gym / Fitness Center,Gluten-free Restaurant,Garden,Convention Center,Clothing Store,Café,Thai Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop
5,0,WC1R 4AG,50200,WC1R 4LR,Argentinian Restaurant,Sandwich Place,Pub,Plaza,Pharmacy,Persian Restaurant,...,Gym / Fitness Center,Gluten-free Restaurant,Garden,Convention Center,Coffee Shop,Clothing Store,Café,Italian Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant
6,0,WC1R 4AH,50200,WC1R 4NA,Argentinian Restaurant,Sushi Restaurant,Sandwich Place,Pub,Plaza,Pharmacy,...,Hotel,Gym / Fitness Center,Gluten-free Restaurant,Garden,Convention Center,Coffee Shop,Clothing Store,Café,Vegetarian / Vegan Restaurant,Thai Restaurant
7,0,WC1R 4AJ,50200,WC1R 4PD,Japanese Restaurant,Sushi Restaurant,Pub,Plaza,Pharmacy,Persian Restaurant,...,Gym / Fitness Center,Gluten-free Restaurant,Garden,Convention Center,Coffee Shop,Clothing Store,Café,Italian Restaurant,Argentinian Restaurant,Sandwich Place
8,0,WC1R 4AL,50200,WC1R 4RL,Argentinian Restaurant,Sushi Restaurant,Sandwich Place,Pub,Plaza,Pharmacy,...,Hotel,Gym / Fitness Center,Gluten-free Restaurant,Garden,Coffee Shop,Clothing Store,Café,Thai Restaurant,Vegetarian / Vegan Restaurant,Convention Center
9,2,WC1R 4AN,50200,WC1V 6AE,Argentinian Restaurant,Sushi Restaurant,Sandwich Place,Pub,Plaza,Pharmacy,...,Hotel,Gym / Fitness Center,Gluten-free Restaurant,Garden,Convention Center,Coffee Shop,Clothing Store,Café,Thai Restaurant,Japanese Restaurant


### Visualize Venue Categories and Income the available locations

#### This is the dataset which will be used to display on a map the venues' post codes and categories. In a pop up to each point, the user will be able to see the post code, the venue's category and the relevant average income.

In [433]:
# Set DataFrame venues_revenue containg the necessary data for visualization

venues_revenue=london_merged_revenue[['Venue Latitude','Venue Longitude','Venue Category','Average Income','PostalCode_Venue']]
venues_revenue.drop_duplicates(subset ="Venue Longitude", inplace = True)

print(venues_revenue.shape)
venues_revenue.head(100)

(26, 5)


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Venue Latitude,Venue Longitude,Venue Category,Average Income,PostalCode_Venue
0,51.518399,-0.120824,Coffee Shop,50200,WC1B 5HA
1,51.517952,-0.119089,Gym / Fitness Center,50200,WC1V 6RD
2,51.517988,-0.120199,Hotel,50200,WC1B 4AA
3,51.519659,-0.117232,Sandwich Place,50200,WC1R 4PD
5,51.519991,-0.118266,Convention Center,50200,WC1R 4RL
7,51.518480,-0.116885,Thai Restaurant,50200,WC1R 4NA
14,51.518838,-0.116870,Italian Restaurant,50200,W C1R
16,51.517320,-0.133661,Juice Bar,54000,W1T 1HT
25,51.519520,-0.117360,Argentinian Restaurant,50200,WC1R 4PD
32,51.519100,-0.115283,Sushi Restaurant,50200,WC1R 4LR


#### Create map visualizing venues' post codes, categories and avegare income in that area

In [432]:
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

# define map
map_new = folium.Map(location=[latitude_L, longitude_L], zoom_start=16)

# add markers to the map
for lat, lon, post, inc, cat in zip(venues_revenue['Venue Latitude'], venues_revenue['Venue Longitude'], venues_revenue['PostalCode_Venue'], venues_revenue['Average Income'], venues_revenue['Venue Category']):
    label = folium.Popup(str(post) + "-" + str(cat) + "-" + str(inc), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='orange',
        fill_opacity=0.7).add_to(map_new)
       
map_new

Note: Due to the lack of diversity in income data, data has not been clustered based on income ranges, but this is another option for better visualization of such parameters should appropriate income data be available.